In [1]:
# Task description:
#У меня есть 985 комментариев о работе компании и пожеланиях клиентов. 
#1. Нужно получить коэффициенты сентиментов по каждому, то есть насколько комментарий 
#положительный или отрицательный. 
#2. Также желательно определить, к каким отделам относятся 
#эти комментарии, чтобы понять, кому их надо брать в работу. 
#3. Задача максимум: ещё и связать 
#коэффициенты сентиментов с количественными оценками, которые клиенты выставляли по 
#отдельным критериям работы компании. Вы такое умеете делать?

In [2]:
import pandas as pd #импорт модуля пандас под именем пд, необходимого для работы с таблицей из экселя
from string import punctuation #импорт объекта для удаления знаков препинания между словами
import os.path

import nltk #импорт пакета nltk для NLP
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords #импорт объекта для удаления лишних (стоп) слов из модуля corpus из пакета nltk для NLP

nltk.download([
    "punkt",
    "stopwords",
])

[nltk_data] Downloading package punkt to /Users/lucifer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lucifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def setup(path: str):# = os.path.abspath(input("Enter path of your Excel document, or press enter and path to folder will be default: "))):
    try:
        survey = pd.read_excel(path,usecols='A,J') #выбираем только колонки A, J для идентификации и анализа
    except OSError:
        if not os.path.isfile(path):
            print("Excel document isn't exist and path will be by default")
            raise
            pd.read_excel("/Users/lucifer/Desktop/20220504_DataScience_Python_NLTK_PROFI.ru_Nelya/Excel2.xlsx", usecols='A,J') #выбираем только колонки A, J для идентификации и анализа
            
setup("/Users/lucifer/Desktop/20220504_DataScience_Python_NLTK_PROFI.ru_Nelya/Excel2.xlsx")

In [ ]:
def GetDataFromDocByKey(document: list, key: str) -> list:
    if len(document) == 0:
        raise ValueError('Argument empty', type(document))
        
    if len(key) == 0:
        raise ValueError('Argument empty', type(key))  
    
    return document[key].astype(str).tolist()

comments = GetDataFromDocByKey(survey, "Комментарий клиента") #создаем массив только с комментариями
clientsIds = GetDataFromDocByKey(survey, "Id шаблона") #создаем массив только с id клиента 

In [ ]:
# TODO: It isn't best code it should be ovewrited to follow SOLID principles  
def matchrus(text: [], alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')) -> bool:
    return not alphabet.isdisjoint(text.lower()) #функция для проверки, если комментарий написан на русском

# TODO: It isn't best code it should be ovewrited to follow SOLID principles 
def matcheng(text: [], alphabet = set('abcdefghijklmnopqrstuvwxyz')) -> bool:
    return not alphabet.isdisjoint(text.lower()) #функция для проверки, если комментарий написан на английском

In [ ]:
def GetCommentsRusOrEng(clientsIds: [], comments: list) -> []:
    if len(clientsIds) == 0:
        raise ValueError('Argument empty', type(clientsIds))
        
    if len(comments) == 0:
        raise ValueError('Argument empty', type(comments))
    
    russiancomments = [] # создаем три пустых массива, чтобы в них записать, соответственно, комментарии на русском
    englishcomments = [] # на английском
    nulcomments = [] # и "нулевые"
    
    for i in range(0, len(clientsIds)): #цикл, который исполняется для каждого id клиента
        array = [] #пустой массив
        array.append(i) #в него записываем порядковый номер строки, начиная с 0
        array.append(clientsIds[i]) #следом в него записываем id клиента
        array.append(comments[i]) #и его комментарий

        # TODO: I think it's possible to move the check to a separate method
        if matchrus(comments[i]):
            russiancomments.append(array) #если комментарий на русском, записываем в массив для руссских комментариев
        elif matcheng(comments[i]):
            englishcomments.append(array) #аналогично для английского
        else:
            nulcomments.append(array) #оставшиеся попадают в "нулевой" массив
            
    return russiancomments, englishcomments, nulcomments

In [ ]:
russiancomments, englishcomments, nulcomments = GetCommentsRusOrEng(clientsIds, comments)

In [ ]:
def PrintComments(comments: [], lang: str) -> None:
    if len(comments) == 0:
        raise ValueError('Argument empty', type(comments))
        
    if len(lang) == 0:
        raise ValueError('Argument empty', type(lang))
    
    print(f"{lang} comments", comments)

#получился массив в массиве на самом деле, но для простоты понимания ниже буду называть массивы в массиве строчками
PrintComments(russiancomments, "Russia")
PrintComments(englishcomments, "English")
PrintComments(nulcomments, "Nul")

print("Amount of Russia comments: ", len(russiancomments)) #печатаем количество строк в каждом массиве
print("Amount of English comments: ", len(englishcomments)) #так их можно сложить и ерепроверить, что сумма сошлась верно
print("Amount of Nul comments: ", len(nulcomments)) #можно это и булевой функцией записать коротко, но пока нужна наглядность для корректировки

In [ ]:
def GetStopwords(lang: str) -> []:
    if len(lang) == 0:
        raise ValueError('Argument empty', type(lang))
    
    return stopwords.words(lang)

russian_stopwords = GetStopwords("russian")
print(russian_stopwords) #печатаем русские лишние (стоп) слова из пакета nltk, чтобы понимать, что будем дальше выкидывать

english_stopwords = GetStopwords("english")
print(english_stopwords) #для английского отдельно

In [ ]:
def GetTokensOrWords(comments, stopwords = None, punctuation = None) -> []:
    if len(comments) == 0:
        raise ValueError('Argument empty', type(comments))
    
    if stopwords is not None and len(stopwords) == 0:
        raise ValueError('Argument empty', type(stopwords))
        
    if punctuation is not None and len(punctuation) == 0:
        raise ValueError('Argument empty', type(punctuation))
    
    tokens = []
    
    for i in range(0, len(comments)): #делим тексты на токены, то есть на отдельные слова, выкидывая знаки препинания и стоп-слова
        text = comments[i][2]
        tokens = nltk.word_tokenize(text.lower())
        if stopwords is not None and punctuation is not None:
            tokens = [token for token in tokens if token not in stopwords and token not in punctuation]
    return tokens

In [ ]:
russianWords = GetTokensOrWords(russiancomments, russian_stopwords, punctuation)
englishWords = GetTokensOrWords(englishcomments, english_stopwords, punctuation)
nulWords = GetTokensOrWords(nulcomments)

In [ ]:
print ("Russian words", russianWords)
print ("English words", englishWords)
print ("Nul words", nulWords)

In [ ]:
def comment_is_positive(comment: str) -> bool:
    if len(comment) == 0:
        raise ValueError('Argument empty', type(comment))
        
    """True if comment has positive compound sentiment, False otherwise."""
    return sia.polarity_scores(comment)["compound"] > 0

def PrintSentiments(sentimentAnalyzer, comments: []) -> None:
    if sentimentAnalyzer is None:
        raise ValueError('Argument empty', type(sentimentAnalyzer))
    
    if len(comments) == 0:
        raise ValueError('Argument empty', type(comments))
    
    for kv_comment in comments:
        value = kv_comment[2]
        print(">", comment_is_positive(value), value)

def SentimentAnalyses(comments: [], lang: str = "Eng") -> None:
    if len(comments) == 0:
        raise ValueError('Argument empty', type(comments))
    
    if len(lang) == 0:
        raise ValueError('Argument empty', type(lang))
    
    #if lang is "Russian" or lang is "Rus":
    if lang == "English" or lang == "Eng":
        sia = SentimentIntensityAnalyzer()
        PrintSentiments(sia, comments)

In [ ]:
SentimentAnalyses(englishcomments)